In [22]:
import cv2
import os
import numpy as np
from scipy.ndimage import label

In [2]:
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

In [21]:
cv2.imread(f'/Users/gauranshurathee/Desktop/shadow_positioning/test_data/nov26/hdri/lonely_road_afternoon_puresky_4k.exr', cv2.IMREAD_UNCHANGED).shape

(2048, 4096, 3)

In [9]:
img_name = [
    f'image{i}'
    for i in [1,2,3,4,5,6,7,8,10,11,12,13]
]
imgs = [
    cv2.imread(f'/Users/gauranshurathee/Desktop/shadow_positioning/test_data/nov26/hdri/image{i}.exr', cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    for i in [1,2,3,4,5,6,7,8,10,11,12,13]
]

In [25]:
for i in range(len(imgs)):
    thres = np.percentile(imgs[i], 99)
    print(f'\n{img_name[i]}:')

    brightness_map = np.sum(imgs[i], axis=2)
    threshold = np.percentile(brightness_map, 98)
    binary_map = brightness_map > threshold

    labeled_array, num_features = label(binary_map)

    max_hdri_intensity = np.max(brightness_map)

    print(num_features)


image1:
20

image2:
22

image3:
14

image4:
18

image5:
37

image6:
7

image7:
12

image8:
35

image10:
6

image11:
1

image12:
1

image13:
5


In [1]:
import os
import numpy as np
import shutil
import trimesh
import pymeshlab
from tqdm import tqdm

In [10]:

TEMP_FILE = 'tmp_largest_component.obj'


mesh = trimesh.load(f'/Users/gauranshurathee/Desktop/shadow_positioning/test_data/dec3_models/result_DSC03648_1024.obj', process=False)


# Find connected components
components = mesh.split()

# Calculate volumes of the components
volumes = np.array([component.volume for component in components])


# Keep the largest component (filtering out smaller components)
largest_mesh = max(components, key=lambda x: x.volume)

# Convert the trimesh object to a temporary .obj file to load into pymeshlab
# output_filename = f'{OUTPUT_FOLDER}/{filename}'
mesh.export(TEMP_FILE)


# Now, use pymeshlab for Laplacian smoothing
ms = pymeshlab.MeshSet()

# Load the largest component from the temporary file
ms.load_new_mesh(TEMP_FILE)

# Apply Laplacian smoothing with default parameters
ms.apply_color_laplacian_smoothing_per_vertex(iteration=40)

# Save the smoothed component back to the output folder
output_filename = f'/Users/gauranshurathee/Desktop/shadow_positioning/test_data/dec3_models/result_DSC03648_1024_smooth.obj'
ms.save_current_mesh(output_filename)

print(f'Saved smoothed mesh to {output_filename}')

# Delete the temporary file after processing
if os.path.exists(TEMP_FILE):
    os.remove(TEMP_FILE)
    print(f'Temporary file {TEMP_FILE} deleted.')


Saved smoothed mesh to /Users/gauranshurathee/Desktop/shadow_positioning/test_data/dec3_models/result_DSC03648_1024_smooth.obj
Temporary file tmp_largest_component.obj deleted.
